In [ ]:
import sys
print(sys.executable)

In [ ]:
import os
import torch
import numpy as np

# CODE FILES HERE
from model_params import get_model_data_tdcvae2
from models.tdcvae2.tdcvae2 import Tdcvae2
from solver import Solver
from dataloader import DataLoader
from directories import Directories, load_all_lungscan_folders, select_lungscan_folders
from plots import plot_losses, plot_gaussian_distributions, plot_vis_tensor, plot_rl_kl, plot_lungscans_samples_grid

# SETTINGS HERE
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # to see the CUDA stack
%matplotlib inline
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
# supress cluttering warnings in solutions
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# setting device on GPU if available, else CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

# Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

In [ ]:
dataset_arg = "lungscans"
data = get_model_data_tdcvae2(dataset_arg)

In [ ]:
# Inits to read all folder names but not yet the data (loaded in DataLoader class)
folders = load_all_lungscan_folders()

In [ ]:
# Select only specific folders from all folder names
selected_folders = ["XhYKtKXNDWQBdy3Ht3hz1VdHo", "Of5p74VhHV8SqP9ERs1boodMq"]
folders = select_lungscan_folders(folders, selected_folders)

In [ ]:
directories = Directories("tdcvae2", dataset_arg.lower(), data["z_dim"], make_dirs=True)
data_loader = DataLoader(directories, data["batch_size"], dataset_arg.lower(), resize=data["resize"], folders=folders)
model = Tdcvae2(data["z_dim"], data["beta"], data["kernel_size_high"], data["kernel_size_low"])
solver = Solver(model, data_loader, data["optimizer"], data["epochs"], data["optim_config"], step_config=data["step_config"],\
                lr_scheduler=data["lr_scheduler"], tdcvae2_mode=True, save_model_state=True, num_samples=8)
solver.main()

In [ ]:
#Uncomment to load a model to continue training.
#solver = torch.load("../results/tdcvae2/lungscans_z=100_35/model_state.pt")
#solver.main()

In [ ]:
#Uncomment to load a trained model for inference.
#solver = torch.load("../results/tdcvae2/lungscans_z=8_4/model_state.pt", map_location='cpu')
#solver.model.eval()

In [ ]:
# Plotting train and test losses for all epochs
plot_losses(solver, solver.train_loss_history["train_loss_acc"], solver.test_loss_history)

In [ ]:
# Plotting the gaussian of z space and some metrics about the space
plot_gaussian_distributions(solver, len(solver.train_loss_history["train_loss_acc"]))

In [ ]:
# Monitoring the reconstruction loss (likelihood lower bound) and KL divergence
DEBUG = 1
if DEBUG:
    for epoch, train_loss, test_loss, rl, kl in zip(solver.train_loss_history["epochs"], \
        solver.train_loss_history["train_loss_acc"], solver.test_loss_history, \
        solver.train_loss_history["recon_loss_acc"], solver.train_loss_history["kl_diverg_acc"]):
        print("epoch: {}, train_loss: {:.2f}, test_loss: {:.2f}, recon. loss: {:.2f}, KL div.: {:.2f}, BETA*KL: {:.5f}".format(
            epoch, train_loss, test_loss, rl, kl, solver.model.beta*kl))
plot_rl_kl(solver, solver.train_loss_history["recon_loss_acc"], solver.train_loss_history["kl_diverg_acc"])

In [ ]:
# Visualize convolutional kernels.
for name, param in solver.model.named_parameters():
    if len(list(param.shape)) == 4:
        plot_vis_tensor(param)

In [ ]:
test_loader = solver.data_loader.get_new_test_data_loader()

In [ ]:
# Visualizing the activations of the convolutions.
x_t, x_next = next(iter(test_loader))
x_t1 = solver.model.encoder.conv1.forward(x_t)
plot_vis_tensor(x_t1, nrow=x_t1.shape[0], padding=1, allkernels=True)
x_t2 = solver.model.encoder.conv2.forward(x_t1)
plot_vis_tensor(x_t2, nrow=x_t2.shape[0], padding=1, allkernels=True)
x_t3 = solver.model.encoder.conv3.forward(x_t2)
plot_vis_tensor(x_t3, nrow=x_t3.shape[0], padding=1, allkernels=True)

In [ ]:
# Producing a grid of samples.
x_t, x_next = next(iter(test_loader))
plot_lungscans_samples_grid(solver, solver.model.decoder, x_t, x_next, solver.model.z_dim, n=4)